# Score Normalization with unsuperviesd info
-------------

z-norm과 s-norm을 구현하였다.

기존에 다른 데이터셋으로 부터 imposter set을 가져와서 score normalization 했던 것에서 벗어나,  
Unsupervised하게 모인 데이터를 가지고 score normalization을 수행

adapt trial의 prediction 기준으로 imposter_embeds를 설정하면 에러 때문에(에러가 꽤 클 경우), imposter set의 score 분포가 깨진다.  
adapte_label을 기준으로 하면 성능이 좋다.

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os, sys
import pickle
import pandas as pd
import numpy as np

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [46]:
from adaptive_sv_system.score_norm_utils import key2df

keys = pickle.load(open("./xvector_embeds/sv_keys.pkl", "rb"))
key_df = key2df(keys)
key2id = {k:v for v, k in enumerate(keys)}

sv_embeds = np.load("./xvector_embeds/sv_embeds.npy")
validation_set = pd.read_pickle("trials/enr306/validation_set.pkl")

In [21]:
def uttrkey_to_id(key2id, keys):
    return [key2id[key] for key in keys]

In [22]:
# from score_norm_utils import uttrkey_to_id
imposter_set = validation_set.sample(n=500)
ood_imposter_embeds = sv_embeds[uttrkey_to_id(key2id, imposter_set.index)]

## Parsing trace then score normalization

TODO: run_trial에 넣어야한다... 너무 느리다.

trace = [adapt_tr, test_tr, ood_tr]  
tr = trial_idxs, scores, label, pred 또는   
tr = trial_idxs, scores, label, pred, enroll_pred  

In [10]:
threshold = 0.56792

In [25]:
base_dir = "adaptive_sv_system/enr306_closedset_out/FS3_ENR3_QUE7/inc_update_neg/"
traces = pickle.load(open(base_dir+"trace.pkl", "rb"))
meta_info = pd.read_pickle(base_dir+"meta_info_df.pkl")

In [47]:
import torch
from adaptive_sv_system.score_norm_utils import eval_cosine_score, eval_z_score, eval_s_score

cos_acc = []
z_acc = []
s_acc = []
for t_idx in range(len(traces)):
    tr = traces[t_idx]
    meta = meta_info.iloc[t_idx]
    adapt_tr, test_tr, ood_tr = tr
    init_enr_idx = meta_info.iloc[t_idx].enr_idxs
    init_enr_embeds = sv_embeds[init_enr_idx]

    adapt_trial_idxs, adapt_scores, adapt_label, adapt_pred, adapt_enroll_pred = adapt_tr
    adapt_embeds = sv_embeds[adapt_trial_idxs.astype(np.int64)]
    adapt_enr_embeds = adapt_embeds[adapt_enroll_pred == 1]
    total_enr_embeds = np.concatenate([init_enr_embeds, adapt_enr_embeds])

    test_trial_idxs, test_scores, test_label, test_pred = test_tr
    test_embeds = sv_embeds[test_trial_idxs.astype(np.int64)]
    test_label = torch.from_numpy(test_label).byte()
    
    enr_embeds = total_enr_embeds
#     enr_embeds = init_enr_embeds
    
#     imposter_embeds = adapt_embeds[adapt_pred==0]
    imposter_embeds = ood_imposter_embeds

    score, pred_acc, eer = eval_cosine_score(
                                             enr_embeds, test_embeds, 
                                             test_label, threshold)
    
    z_score, z_pred_acc, z_eer = eval_z_score(
                                              enr_embeds, test_embeds, 
                                              test_label, threshold, 
                                              imposter_embeds)
    
    s_score, s_pred_acc, s_eer = eval_s_score(
                                              enr_embeds, test_embeds,
                                              test_label, threshold,
                                              imposter_embeds)
    
    
#     print("cos_acc:{}\nz_acc:{}\ns_acc:{}".format(pred_acc, z_pred_acc, s_pred_acc))
#     print("="*20)
    cos_acc += [pred_acc]
    z_acc += [z_pred_acc]
    s_acc += [s_pred_acc]


In [48]:
# only init_enr_embeds
# ood_imposter
print("cos_acc:{:.4f}\nz_acc:{:.4f}\ns_acc:{:.4f}".format(
    np.mean(cos_acc), np.mean(z_acc), np.mean(s_acc)))

cos_acc:0.9713
z_acc:0.9713
s_acc:0.5535


In [49]:
# only init_enr_embeds
# adapt_imposter
print("cos_acc:{:.4f}\nz_acc:{:.4f}\ns_acc:{:.4f}".format(
    np.mean(cos_acc), np.mean(z_acc), np.mean(s_acc)))

cos_acc:0.9713
z_acc:0.9713
s_acc:0.5535


In [50]:
# total_enr_embeds
# adapt_imposter
print("cos_acc:{:.4f}\nz_acc:{:.4f}\ns_acc:{:.4f}".format(
    np.mean(cos_acc), np.mean(z_acc), np.mean(s_acc)))

cos_acc:0.9713
z_acc:0.9713
s_acc:0.5535


In [40]:
adapt_embeds[adapt_pred==0].shape

(364, 512)